In [ ]:
using Plots
using LaTeXStrings
using Statistics
using ProgressMeter

In [ ]:
default(xtickfont=font(12), ytickfont=font(12), guidefont=font(12),
    legendfontsize=10, lw=2, ms=8)

In [ ]:
include("bayesinverse.jl")

In [ ]:
N = 64;
γ = 5e-4;
x = LinRange(0, 1, N + 1) |> collect;

Δx = x[2] - x[1];
x_mid = x[1:end-1] .+ Δx / 2;

# recover on particular points
x_data = 0:0.1:1;
x_data = x_data[2:end-1] |> collect;

ut(x) = x * (1 - x);
at(x) = exp(ut(x));
f(x) = x^2;
p = solve_bvp(at.(x_mid), f.(x[2:end-1]), Δx);

# generate noisy data
Random.seed!(500); # set a seed for reproducibility
p_spl = LinearInterpolation(p, x);
y_data = @. p_spl(x_data) + γ * randn()


In [ ]:
plot(x, p, label="Truth")
scatter!(x_data, y_data,label="Data")
xlabel!(L"x")

In [ ]:
function loss(u)
    u_int = LinearInterpolation(u, x)
    p = solve_bvp(exp.(u_int.(x_mid)), f.(x[2:end-1]), Δx)
    p_int = LinearInterpolation(p, x)
    l = 0.5 / γ^2 * sum(abs2, p_int(x_data) - y_data)
    return l
end

In [ ]:
u = zeros(N+1);
v = zeros(N+1);
n_iters = 10^6;
# samples = zeros(n_iters, N+1);
u_mean = zeros(N+1);
a_mean = zeros(N+1);
u2_mean = zeros(N+1);
a2_mean = zeros(N + 1);
accept = zeros(n_iters);

Φu = loss(u);
β = 0.1;
Random.seed!(500)
@showprogress for j in 1:n_iters
    ξ = sample_field_fft(N);
    v .= sqrt(1 - β^2) * u + β * ξ
    Φv = loss(v);
    a = min(1, exp(Φu - Φv))
    ζ = rand()
    if ζ < a
        accept[j] = 1;
        @. u = v
        Φu = Φv
    end
    @. u_mean += u;
    @. u2_mean +=u^2;
    @. a_mean += exp(u);
    @. a2_mean += exp(2*u);

end
u_mean /=n_iters;
u2_mean /=n_iters;
u_var =@. max(u2_mean - u_mean^2, 0);

a_mean /= n_iters;
a2_mean /= n_iters;
a_var = @. max(a2_mean - a_mean^2, 0);

In [ ]:
@show mean(accept);

In [ ]:
plot(x, u_mean, ribbon=sqrt.(u_var), label="Estimate")
plot!(x, ut.(x), label="Truth")
xlabel!(L"x")
ylabel!(L"u")

In [ ]:
plot(x, a_mean, ribbon=sqrt.(a_var), label="Estimate")
plot!(x, at.(x), label="Truth")
xlabel!(L"x")
ylabel!(L"a")